In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_absolute_error

import xgboost as xgb
import lightgbm as lgb
import catboost as cat

print('done!')
from warnings import filterwarnings
filterwarnings("ignore")

In [ ]:
path = "../input/ventilator-pressure-prediction/"
data = {
    'train' : path + "train.csv",
    'test'  : path + "test.csv",
    'sample': path + "sample_submission.csv"
}

train = pd.read_csv(data["train"])
test  = pd.read_csv(data["test"])
sample= pd.read_csv(data["sample"])


In [ ]:
X = train.copy()
y = X.pop("pressure")

kf = KFold(n_splits = 5, shuffle=True, random_state=42)

final_mae = []
test_preds= []

for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    
    xtrain, ytrain = X.iloc[trn_idx], y.iloc[trn_idx]
    xvalid, yvalid = X.iloc[val_idx], y.iloc[val_idx]
    
    params={
        'n_estimators': 20000,        
        'iterations': 6448,
        'bootstrap_type': 'MVS',
        'od_wait': 1915,
        'learning_rate': 0.7273356024168018,
        'reg_lambda': 75.31560025945122,
        'random_strength': 39.64827863323453,
        'max_depth': 12,
        'min_data_in_leaf': 2,
        'leaf_estimation_iterations': 15,
        'colsample_bytree': 0.8373854007651275,
        'colsample_bynode': 0.8162517464386975,
        'colsample_bylevel': 0.9585701093287505,
        'subsample': 0.9287668460127163,
        'min_child_weight': 48,
        'max_bin': 141,
        
        "predictor": "gpu_predictor",
        'tree_method' : 'gpu_hist', 
        'gpu_id': 0
      }
    
    model = xgb.XGBRegressor(**params)
    model.fit(xtrain, ytrain)
    
    pred = model.predict(xvalid)
    test_pred = model.predict(test)
    
    test_preds.append(test_pred)
    mae  = mean_absolute_error(pred, yvalid)
    print(f"fold : {fold}, mae: {mae}")
    final_mae.append(mae)
    
print(f"final mae: {np.mean(final_mae)}\n")
    

In [ ]:
final_test_score = np.column_stack(test_preds)
final_test_score = final_test_score.mean(axis=1)
output = sample.copy()
output.pressure = final_test_score
output.to_csv("./submission.csv", index=False)

In [ ]:
output.head()